In [160]:
pip install pycryptodome

Note: you may need to restart the kernel to use updated packages.


In [161]:
import uuid
import hashlib
import numpy as np
import base64
from Crypto.Cipher import AES
import os

### Reading the file

In [162]:
filename="file.pdf"

In [163]:
extension=filename.split('.')[1]

In [164]:
with open(filename, mode="rb") as file:
    data = file.read()

In [165]:
file_length=len(data)
file_length

190766

### Encryption

#### Calculating number of blocks based on file size

In [166]:
if file_length >= 1024**3:
    blocks=((file_length/1024**3)*8)
else:
    blocks=8

#### Dividing the file into blocks

In [167]:
file_blocks=[]
block_size=file_length/blocks
temp=data

In [168]:
block_size=int(block_size)
start=0
end=block_size

In [169]:
for i in range(0,blocks-1):
    file_blocks.append(temp[start:end])
    start=end
    end=end+block_size
    
file_blocks.append(temp[start:file_length])

#### Generating Pseudo random numbers using  Linear Congruence Method

In [170]:
Xo=0
m=file_length
a=int(file_length/10)
c=int(file_length/blocks)
length=blocks
random_numbers=[0]*length
random_numbers[0] = Xo

for i in range(1, length):
    random_numbers[i] = ((random_numbers[i - 1] * a) +c) % m
random_numbers.sort()
random_numbers

[0, 16787, 23845, 104921, 107149, 131245, 147309, 170735]

#### Generating UUID String

In [171]:
uuid_str=str(uuid.uuid4())
uuid_str=uuid_str.replace("-","")

#### Generating MD5 Strings

In [172]:
hash_str=hashlib.md5(uuid_str.encode())
hash_str=hash_str.hexdigest()

#### Dividing UUID & MD5 Strings

In [173]:
start=0
end=4
uuid_blocks=[]
md5_blocks=[]

for i in range(0,blocks):
    uuid_blocks.append(uuid_str[start:end])
    md5_blocks.append(hash_str[start:end])
    start=end
    end=end+4

#### Initializing Data Matrix

In [174]:
data_matrix= [[b'']*int(4) for i in range(blocks)]

#### Encrypting data using matrix

In [175]:
for i in range(0,blocks):
    data_matrix[i][0]=md5_blocks[i].encode()
    data_matrix[i][1]=bytes(random_numbers[i])
    data_matrix[i][2]=file_blocks[i]+b'100'
    data_matrix[i][3]=uuid_blocks[i].encode()

#### Converting Matrix into String

In [176]:
matrix_string=b''
for i in range(0,blocks):
    for j in range(0,4):
        matrix_string=matrix_string+data_matrix[i][j]+b'/t'

In [177]:
data_string=matrix_string

#### Encrypting String using AES

##### Padding String to make sure it is a multiple of 16

In [178]:
padding=b' '*(16-(len(data_string)%16))
data_string+=padding

##### Generating key and encryting the data

In [179]:
iv = os.urandom(16)
key=os.urandom(16)
encryption_cipher=AES.new(key,AES.MODE_CBC,iv)
encrypted_data=encryption_cipher.encrypt(data_string)

##### Storing the encrypted file

In [180]:
with open("encrypted."+extension, mode="wb") as file:
      file.write(encrypted_data)

##### Saving Key and IV in a file

In [181]:
with open("key.env", mode="wb") as file:
    file.write(key)
    file.write(iv)

### Decryption

#### Reading Key and IV from a file

In [182]:
with open("key.env", mode="rb") as file:
    content=bytes(file.read())
    
key2=content[0:16]
iv2=content[16:32]

#### Reading content from file

In [183]:
with open("encrypted."+extension, mode="rb") as file:
    encrypted_data=file.read()

#### Decrypting String using AES

In [184]:
decryption_cipher=AES.new(key2,AES.MODE_CBC,iv2)
data=decryption_cipher.decrypt(encrypted_data)

##### Removing the padding

In [185]:
data=data.strip()

#### Decrypting data from matrix

##### Converting String into matrix

In [186]:
matrix_data=data.split(b'/t')
x=0
dec_matrix=[[b'']*int(4) for i in range(blocks)]
for i in range(0,blocks):
    for j in range(0,4):
        dec_matrix[i][j]=matrix_data[x]
        x=x+1

In [190]:
##### Decrypting

In [187]:
dec_file_blocks=[]
for i in range(0,blocks):
    dec_file_blocks.append(dec_matrix[i][2])
    dec_file_blocks[i]=dec_file_blocks[i][0:len(dec_file_blocks[i])-3]

In [188]:
out=b''
for i in dec_file_blocks:
    out=out+i

#### Writing the decrypted content to file

In [189]:
with open("decrypted."+extension, mode="wb") as file:
    file.write(out)